# Superstore Analysis: Uncovering a Profitability Crisis 

## Section 1. Data Loading

**Goal:** Load the raw CSV data into a SQLite database for querying and analysis.

This step creates the `superstore.db` database and the `superstore_raw` table.

In [1]:
import pandas as pd
import sqlite3

# Load superstore.csv dataset
superstore_df = pd.read_csv('superstore.csv')
superstore_df.head()

,Category,City,Country,Customer.ID,Customer.Name,Discount,Market,记录数,Order.Date,Order.ID,...,Sales,Segment,Ship.Date,Ship.Mode,Shipping.Cost,State,Sub.Category,Year,Market2,weeknum
0,Office Supplies,Los Angeles,United States,LS-172304,Lycoris Saunders,0.0,US,1,2011-01-07 00:00:00.000,CA-2011-130813,...,19,Consumer,2011-01-09 00:00:00.000,Second Class,4.37,California,Paper,2011,North America,2
1,Office Supplies,Los Angeles,United States,MV-174854,Mark Van Huff,0.0,US,1,2011-01-21 00:00:00.000,CA-2011-148614,...,19,Consumer,2011-01-26 00:00:00.000,Standard Class,0.94,California,Paper,2011,North America,4
2,Office Supplies,Los Angeles,United States,CS-121304,Chad Sievert,0.0,US,1,2011-08-05 00:00:00.000,CA-2011-118962,...,21,Consumer,2011-08-09 00:00:00.000,Standard Class,1.81,California,Paper,2011,North America,32
3,Office Supplies,Los Angeles,United States,CS-121304,Chad Sievert,0.0,US,1,2011-08-05 00:00:00.000,CA-2011-118962,...,111,Consumer,2011-08-09 00:00:00.000,Standard Class,4.59,California,Paper,2011,North America,32
4,Office Supplies,Los Angeles,United States,AP-109154,Arthur Prichep,0.0,US,1,2011-09-29 00:00:00.000,CA-2011-146969,...,6,Consumer,2011-10-03 00:00:00.000,Standard Class,1.32,California,Paper,2011,North America,40


In [2]:
# Connect to SQLite database and create a file
db_conn = sqlite3.connect('superstore.db')

# Load the dataframe into the database
superstore_df.to_sql('superstore_raw', db_conn, if_exists='replace',index=False) # name new table as superstore_raw
db_conn.close()

In [3]:
# Checking to see if the dataframe is in the database correctly
# Reconnect to db
db_conn = sqlite3.connect('superstore.db')

# Run a query to list all the tables in the db
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Run query using pandas and return a df
table_list = pd.read_sql_query(query, db_conn)
print("Tables in the database:")
print(table_list['name'].to_string(index=False)) 

db_conn.close()

Tables in the database:
superstore_raw


## Section 2. Data Cleaning & Transformation with SQL

**Goal:** Use SQL to remove any redudant columns and prepare a clean dataset for analysis.

This step uses the `data_cleaning.sql` script to create the clean `superstore_clean` table.

In [4]:
# Read in the data_cleaning.sql file 
import sqlite3

db_conn = sqlite3.connect('superstore.db')
db_cursor = db_conn.cursor()

with open('data_cleaning.sql', 'r') as file:
    sql_script = file.read()

# Split the script and execute them one by one
indiv_script = sql_script.split(';') # Split the script using a semi-colon as the delimiter

for action in indiv_script:
    action = action.strip() # Removes whitespace
    if action == '':
        continue
    try:
        db_cursor.execute(action)
    except Exception as e:
        print("Error")
        break # Stop if theres an error

db_conn.close()


In [5]:
db_conn = sqlite3.connect('superstore.db')

# Checking to see that there is now 2 databases
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", db_conn)
print("Tables in the database:")
print(tables)

# Checking that the '记录数' column is gone by looking at all the columns
print("\nColumns in superstore_clean:")
columns = pd.read_sql_query("PRAGMA table_info(superstore_clean);", db_conn)
print(columns['name'].to_string(index=False))

db_conn.close()

Tables in the database:
               name
0    superstore_raw
1  superstore_clean

Columns in superstore_clean:
      Category
          City
       Country
   Customer.ID
 Customer.Name
      Discount
        Market
    Order.Date
      Order.ID
Order.Priority
    Product.ID
  Product.Name
        Profit
      Quantity
        Region
        Row.ID
         Sales
       Segment
     Ship.Date
     Ship.Mode
 Shipping.Cost
         State
  Sub.Category
          Year
       Market2
       weeknum


## Section 3. Business Analysis with SQL

### 3.1 - High-Level Profitability Overview

**Question:** Is our most profitable category also our most efficient, or are there hidden outliers?

This query calculates total sales, profit, and profit margin for each product category to identify overall performance and potential problem areas.

In [6]:
db_conn = sqlite3.connect('superstore.db')

# Create a query that shows which category has the highest profit
profit_query = """
SELECT
    Category,
    SUM(Sales) AS Total_Sales,
    ROUND(SUM(Profit), 2) AS Total_Profit,
    ROUND((SUM(Profit) / SUM(Sales)) * 100, 2) AS Profit_Margin_Percent
FROM superstore_clean
GROUP BY Category
ORDER BY Total_Profit DESC;
"""

# Load the SQL query into a dataframe so we can print it
profit_df = pd.read_sql_query(profit_query, db_conn)
print(profit_df)


          Category  Total_Sales  Total_Profit  Profit_Margin_Percent
0       Technology      4744691     663778.73                  13.99
1  Office Supplies      3787330     518473.83                  13.69
2        Furniture      4110884     285204.72                   6.94


In [7]:
# Printing out the dataframe and sorting by different values to see which Category performs best
profit_sorted = profit_df.sort_values(by=['Total_Profit'], ascending=False)
sales_sorted = profit_df.sort_values(by=['Total_Sales'], ascending=False)
margin_sorted = profit_df.sort_values(by=['Profit_Margin_Percent'], ascending=False)

print("Total Profits (Highest to Smallest)")
print(profit_sorted)

print("\nTotal Sales (Highest to Smallest)")
print(sales_sorted)

print("\nProfit Margin Percent (Highest to Smallest)")
print(margin_sorted)


Total Profits (Highest to Smallest)
          Category  Total_Sales  Total_Profit  Profit_Margin_Percent
0       Technology      4744691     663778.73                  13.99
1  Office Supplies      3787330     518473.83                  13.69
2        Furniture      4110884     285204.72                   6.94

Total Sales (Highest to Smallest)
          Category  Total_Sales  Total_Profit  Profit_Margin_Percent
0       Technology      4744691     663778.73                  13.99
2        Furniture      4110884     285204.72                   6.94
1  Office Supplies      3787330     518473.83                  13.69

Profit Margin Percent (Highest to Smallest)
          Category  Total_Sales  Total_Profit  Profit_Margin_Percent
0       Technology      4744691     663778.73                  13.99
1  Office Supplies      3787330     518473.83                  13.69
2        Furniture      4110884     285204.72                   6.94


### 3.2 - Investigating the Outlier: The Furniture Category

**Question:** Why does the Furniture category have such low margins than Technology and Office Supplies?

A look into Furniture sub-categories reveals the specific product line responsible for dragging down the entire category's performance.

In [8]:
# Create a query that looks into the furniture category
furniture_query = """
SELECT 
    "Sub.Category",
    ROUND(SUM(Sales), 2) AS Total_Sales,
    ROUND(SUM(Profit), 2) AS Total_Profit,
    ROUND((SUM(Profit) / SUM(Sales)) * 100, 2) AS Profit_Margin_Percent
FROM superstore_clean
WHERE Category = 'Furniture'
GROUP BY "Sub.Category"
ORDER BY Profit_Margin_Percent ASC; 
"""

# Load the SQL query into a dataframe so we can print it
furniture_df = pd.read_sql_query(furniture_query, db_conn)
print(furniture_df.to_string(index=False))

Sub.Category  Total_Sales  Total_Profit  Profit_Margin_Percent
      Tables     757034.0     -64083.39                  -8.47
      Chairs    1501682.0     140396.27                   9.35
   Bookcases    1466559.0     161924.42                  11.04
 Furnishings     385609.0      46967.43                  12.18


### 3.3 - Geographic Analysis: Pinpointing the Problem

**Question:** Is the profitability issue with Tables a universal problem, or is it only present in specific markets?

This analysis moves from identifying the "what" (Tables) to the "where" (specific markets), enabling targeted recommendations.

In [9]:
# Create a query that looks further into tables, specifically region
furniture_regions_query = """
SELECT 
    Region,
    "Sub.Category",
    ROUND(SUM(Sales), 2) AS Total_Sales,
    ROUND(SUM(Profit), 2) AS Total_Profit,
    ROUND((SUM(Profit) / SUM(Sales)) * 100, 2) AS Profit_Margin_Percent
FROM superstore_clean
WHERE "Sub.Category" = 'Tables'
GROUP BY Region, "Sub.Category"
ORDER BY Profit_Margin_Percent ASC;
"""

# Load the SQL query into a dataframe so we can print it
furniture_regions_df = pd.read_sql_query(furniture_regions_query, db_conn)
print(furniture_regions_df.to_string(index=False))

        Region Sub.Category  Total_Sales  Total_Profit  Profit_Margin_Percent
Southeast Asia       Tables      52452.0     -18618.31                 -35.50
          East       Tables      39142.0     -11025.38                 -28.17
         South       Tables     104894.0     -27012.32                 -25.75
       Central       Tables     133126.0     -21550.11                 -16.19
    North Asia       Tables      54299.0      -5470.58                 -10.07
       Oceania       Tables      66833.0       -229.72                  -0.34
     Caribbean       Tables      23660.0         63.39                   0.27
          West       Tables      84755.0       1482.61                   1.75
          EMEA       Tables      39331.0       2764.25                   7.03
  Central Asia       Tables      51512.0       4189.85                   8.13
         North       Tables      71649.0       7012.06                   9.79
        Africa       Tables      34531.0       4010.68          

In [10]:
# Create a query that looks further into tables, specifically market
furniture_market_query = """
SELECT 
    Market, Region
    "Sub.Category",
    ROUND(SUM(Sales), 2) AS Total_Sales,
    ROUND(SUM(Profit), 2) AS Total_Profit,
    ROUND((SUM(Profit) / SUM(Sales)) * 100, 2) AS Profit_Margin_Percent
FROM superstore_clean
WHERE "Sub.Category" = 'Tables'
GROUP BY Market, Region, "Sub.Category"
ORDER BY Profit_Margin_Percent ASC;
"""

# Load the SQL query into a dataframe so we can print it
furniture_market_df = pd.read_sql_query(furniture_market_query, db_conn)
print(furniture_market_df.to_string(index=False))

Market   Sub.Category  Total_Sales  Total_Profit  Profit_Margin_Percent
    EU          South      15141.0      -8974.06                 -59.27
  APAC Southeast Asia      52452.0     -18618.31                 -35.50
 LATAM          South      45834.0     -13415.20                 -29.27
    US           East      39142.0     -11025.38                 -28.17
    EU        Central      59384.0     -15320.58                 -25.80
    US          South      43919.0      -4623.06                 -10.53
  APAC     North Asia      54299.0      -5470.58                 -10.07
    US        Central      39152.0      -3559.65                  -9.09
 LATAM        Central      34590.0      -2669.88                  -7.72
  APAC        Oceania      66833.0       -229.72                  -0.34
 LATAM      Caribbean      23660.0         63.39                   0.27
    US           West      84755.0       1482.61                   1.75
  EMEA           EMEA      39331.0       2764.25                

### 3.4 - Root Cause Analysis: The Discounting Strategy

**Question:** What is driving the massive losses in these specific markets? Is there a correlation with discounting levels?

The final query tests the hypothesis that excessive discounting is the primary cause of the negative profit margins, providing the "why" behind the crisis

In [11]:
# Create a query that looks further into tables, specifically discounts
furniture_discounts_query = """
SELECT 
    Market,
    Region,
    ROUND(AVG(Discount) * 100, 2) AS Avg_Discount_Percent,
    ROUND(SUM(Sales), 2) AS Total_Sales,
    ROUND(SUM(Profit), 2) AS Total_Profit
FROM superstore_clean
WHERE "Sub.Category" = 'Tables'
GROUP BY Market, Region
ORDER BY Avg_Discount_Percent DESC;
"""

# Load the SQL query into a dataframe so we can print it
furniture_discounts_df = pd.read_sql_query(furniture_discounts_query, db_conn)
print(furniture_discounts_df.to_string(index=False))

db_conn.close()


Market         Region  Avg_Discount_Percent  Total_Sales  Total_Profit
    EU          South                 53.16      15141.0      -8974.06
  APAC Southeast Asia                 48.00      52452.0     -18618.31
 LATAM          South                 42.90      45834.0     -13415.20
    EU        Central                 39.17      59384.0     -15320.58
    US           East                 37.38      39142.0     -11025.38
 LATAM        Central                 35.00      34590.0      -2669.88
  APAC     North Asia                 32.62      54299.0      -5470.58
 LATAM      Caribbean                 28.52      23660.0         63.39
  APAC        Oceania                 27.36      66833.0       -229.72
    US        Central                 26.25      39152.0      -3559.65
    US          South                 22.25      43919.0      -4623.06
 LATAM          North                 20.00      40794.0       3715.84
    US           West                 20.00      84755.0       1482.61
  APAC

## 4. Conclusion & Strategic Recommendations

### Summary of Findings:

1.  **The Core Problem:** The Tables sub-category is operating at a significant loss (-8.47% margin), costing the company over $64,000.
2.  **Geographic Concentration:** These losses are highly concentrated in specific markets, notably **EU South** (-59.27% margin) and **APAC Southeast Asia** (-35.50% margin).
3.  **Root Cause:** A direct correlation was found between excessive discounting (averaging 50% in the worst-performing markets) and catastrophic profit losses.

### Strategic Recommendations:

1.  **Immediate Action:** Implement a strict maximum discount threshold for Tables in the EU South and APAC SEA markets to immediately stop the financial losses.
2.  **Sales Strategy Review:** Launch a training program for sales teams in these regions focused on value-based selling. Simultaneously, review sales targets and commissions to ensure they incentivize profitability, not just revenue.
3.  **Long-Term Strategy:** Conduct a full review of the Tables product line in these markets to assess its long-term viability and explore alternatives like cost reduction or product substitution.